In [1]:
import pandas as pd
import statsmodels.formula.api as smf

In [2]:
df_1 = pd.read_csv("homework_12.1.csv")
df_1.head(5)

,Unnamed: 0,Y,Time,Group
0,0,-0.232900,-1.193204,0
1,1,2.848846,-1.607748,1
2,2,0.550209,-0.269793,0
3,3,2.198280,7.743730,0
4,4,4.111044,-4.244359,1


In [3]:
df_1["treated"] = (df_1['Group']==1).astype(int)
df_1["post"] = (df_1["Time"] > 0).astype(int)
df_1["did"] = df_1["treated"] * df_1["post"]

model = smf.ols("Y~treated + post + did", data = df_1).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.9802,0.016,61.953,0.000,0.949,1.011
treated,2.0420,0.036,56.304,0.000,1.971,2.113
post,1.5130,0.022,67.722,0.000,1.469,1.557
did,0.9440,0.051,18.620,0.000,0.845,1.043


In [4]:
df_2 = pd.read_csv("homework_12.2.csv")
df_2.head(5)

,Unnamed: 0,Y,Time,Group
0,0,0.283614,-3.614416,0
1,1,3.181495,-3.064196,0
2,2,1.401619,-4.109261,0
3,3,2.019632,-0.706696,1
4,4,1.032628,-1.806227,0


In [5]:
# q2
# Filter data before treatment (Time < 0)
df_pre = df_2[df_2["Time"] < 0].copy()
# interaction term
df_pre["interaction"] = df_pre["Group"] * df_pre["Time"]

model = smf.ols("Y~Group + Time + interaction", data=df_pre).fit()
t_value = model.tvalues["interaction"]
t_value


np.float64(5.329379663924038)

In [6]:
# q3
import numpy as np
import statsmodels.api as sm
def generate_ar1_errors(n, rho, std=1):
    errors = np.zeros(n)
    errors[0] = np.random.normal(0, std)
    for t in range(1, n):
        errors[t] = rho * errors[t-1] + np.random.normal(0, np.sqrt(1 - rho ** 2) * std)
    return errors

n = 10000
rho = 0.8
std = 1
n_simulations = 200
X_sample = generate_ar1_errors(n, rho, std)
noise_sample = generate_ar1_errors(n, rho, std)
Y_sample = 2 * X_sample + noise_sample

X_const = sm.add_constant(X_sample)
model_sample =  sm.OLS(Y_sample, X_const).fit()
basic_se = model_sample.bse[1]

# simulationi to estimate std of coefficient estimates
coef_estimates = []
for _ in range(n_simulations):
    X = generate_ar1_errors(n, rho, std)
    noise = generate_ar1_errors(n, rho, std)
    Y = 2 * X + noise
    X_const = sm.add_constant(X)
    model = sm.OLS(Y, X_const).fit()
    coef_estimates.append(model.params[1])

simulated_se=np.std(coef_estimates)
simulated_se


np.float64(0.021381920494109592)

In [11]:
np.random.seed(0)
X1 = np.random.normal(0, 1, 1000)
X2 = np.random.normal(0, 1, 1000) + X1
X3 = np.random.normal(0, 1, 1000) + 2 * X2

X_ = np.column_stack([X2, X3])
X_ = sm.add_constant(X_)
model = sm.OLS(X1, X_).fit()
R2 = model.rsquared
VIF = 1/(1-R2)
print(VIF)


1.9783742480901254
